In [1]:
import branca
import clusters_builder
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import importlib
import json
import numpy as np
import pandas as pd
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import requests
import selenium.webdriver

In [2]:
def validate_country_name(df, col_name):
    # according to https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json
    df.loc[df[col_name]=='US',col_name]='United States of America'
    df.loc[df[col_name]=='Congo (Kinshasa)',col_name]='Republic of the Congo'
    df.loc[df[col_name]=='Congo (Brazzaville)',col_name]='Democratic Republic of the Congo'
    df.loc[df[col_name]=='Serbia',col_name]='Republic of Serbia'
    df.loc[df[col_name]=='Czechia',col_name]='Czech Republic'
    df.loc[df[col_name]=='Taiwan*',col_name]='Taiwan'
    df.loc[df[col_name]=='North Macedonia',col_name]='Macedonia'

In [3]:
def my_color_function(feature, df_dict, colorscale):
    if not feature['properties']['ADMIN']  in df_dict:
        return '#ffffff'
    return colorscale(df_dict[feature['properties']['ADMIN']])

In [4]:
def create_colorscale(n):
    colorscale=branca.colormap.linear.YlOrRd_09.scale(0,n)
    colorscale=colorscale.to_step(n)
    colorscale.caption='Cluster id'
    return colorscale

In [5]:
def get_geo_json(clean_data):
    country_shapes='https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson'
    geo_json=json.loads(requests.get(country_shapes).text)
    for i in range(len(geo_json['features'])):
        name=geo_json['features'][i]['properties']['ADMIN']
        for column_name in ['New cases', 'New deaths', 'New recovered']:
            geo_json['features'][i]['properties'][column_name]=0
            if name in clean_data['Country/Region'].values:
                geo_json['features'][i]['properties'][column_name]=\
                int(clean_data[clean_data['Country/Region']==name][column_name].values[0])
        
    return geo_json

In [6]:
def create_map(geo_json,df_dict, n, date):
    colorscale=create_colorscale(n)
    m=folium.Map(tiles="cartodbpositron", zoom_start=2,location=[40., 10.],zoom_control=False, max_bounds=True)
    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': my_color_function(feature, df_dict, colorscale),
            'fillOpacity': 0.9,
            'color' : 'black',
            'weight' : 1,
            'dashArray' : '1,1'
            },
        name='COVID CLUSTERS',
        tooltip=folium.features.GeoJsonTooltip(
            fields=['ADMIN', 'New cases', 'New deaths', 'New recovered'],
            aliases=['Country', 'New cases', 'New deaths', 'New recovered'])
        ).add_to(m)
    m.add_child(colorscale)
    
    
    title_html = '''
                 <h3 align="center" style="font-size:12px"><b>{}</b></h3>
                 '''.format(date)   
    m.get_root().html.add_child(folium.Element(title_html))
    
    m.save('covid.html')

In [7]:
def save_as_png(date):
    driver = selenium.webdriver.Chrome()
    driver.get("file:///C:/Users/DELL/Covid_clusters/covid.html")
    driver.save_screenshot('pictures.v2/'+date+'.png')
    driver.quit()

In [8]:
def save_map(builder,date, n):
    clusters=builder.get_clusters(date)
    print([len(x) for x in clusters])
    clean_data=builder.data_all_days.loc[builder.data_all_days['Date']==date].copy()
    
    rows=[[country,i] for i in range(n) for country in clusters[i]]
    clusters_dict={_id: rows[_id] for _id in range(builder.n_vert) }
    
    df=pd.DataFrame.from_dict(clusters_dict,orient='index', columns=['CountryName', 'Cluster'])

    validate_country_name(df,'CountryName')
    validate_country_name(clean_data, 'Country/Region')
    df_dict=df.set_index('CountryName')['Cluster']
    geo_json=get_geo_json(clean_data)
    create_map(geo_json, df_dict, n, date)
    save_as_png(date)

In [9]:
importlib.reload(clusters_builder)
n=5
builder=clusters_builder.Clusters_Builder(n_clusters=n)
dates=sorted(list(set(builder.data_all_days['Date'].values)))
for date in dates:
    #if date<'2020-03-15':
     #   continue
    save_map(builder, date, n)
    print(date)
    if date>='2020-03-14':
        break

[180, 4, 1, 1, 1]
2020-01-22
[3, 180, 2, 1, 1]
2020-01-23
[180, 1, 1, 4, 1]
2020-01-24
[180, 2, 3, 1, 1]
2020-01-25
[182, 1, 1, 2, 1]
2020-01-26
[179, 1, 1, 5, 1]
2020-01-27
[181, 1, 3, 1, 1]
2020-01-28
[180, 1, 3, 2, 1]
2020-01-29
[183, 1, 1, 1, 1]
2020-01-30
[183, 1, 1, 1, 1]
2020-01-31
[181, 3, 1, 1, 1]
2020-02-01
[180, 1, 4, 1, 1]
2020-02-02
[180, 2, 3, 1, 1]
2020-02-03
[178, 4, 2, 2, 1]
2020-02-04
[180, 4, 1, 1, 1]
2020-02-05
[180, 4, 1, 1, 1]
2020-02-06
[1, 182, 1, 2, 1]
2020-02-07
[1, 183, 1, 1, 1]
2020-02-08
[1, 183, 1, 1, 1]
2020-02-09
[180, 4, 1, 1, 1]
2020-02-10
[1, 182, 2, 1, 1]
2020-02-11
[1, 176, 8, 1, 1]
2020-02-12
[1, 180, 4, 1, 1]
2020-02-13
[180, 4, 1, 1, 1]
2020-02-14
[182, 1, 2, 1, 1]
2020-02-15
[1, 180, 1, 4, 1]
2020-02-16
[180, 4, 1, 1, 1]
2020-02-17
[180, 1, 4, 1, 1]
2020-02-18
[180, 4, 1, 1, 1]
2020-02-19
[180, 4, 1, 1, 1]
2020-02-20
[180, 3, 1, 2, 1]
2020-02-21
[180, 4, 1, 1, 1]
2020-02-22
[180, 1, 1, 4, 1]
2020-02-23
[182, 2, 1, 1, 1]
2020-02-24
[180, 4, 1, 1,

# TESTING

In [91]:
clusters=builder.get_clusters('2020-02-01')

187
built edge list
have clusters


In [97]:
date='2020-02-01'
clean_data=builder.data_all_days.loc[builder.data_all_days['Date']==date].copy()
clusters=sorted(clusters, key=lambda my_set: clean_data[clean_data['Country/Region'].isin(my_set)]['New cases'].mean())

In [101]:
clusters_dict={_id: rows[_id] for _id in range(builder.n_vert) }

{0: ['Azerbaijan', 0], 1: ['Guinea-Bissau', 0], 2: ['Botswana', 0], 3: ['Finland', 0], 4: ['Greenland', 0], 5: ['Colombia', 0], 6: ['Gabon', 0], 7: ['Dominican Republic', 0], 8: ['Canada', 0], 9: ["Cote d'Ivoire", 0], 10: ['Algeria', 0], 11: ['Equatorial Guinea', 0], 12: ['Comoros', 0], 13: ['Haiti', 0], 14: ['Cameroon', 0], 15: ['Hungary', 0], 16: ['Central African Republic', 0], 17: ['Burma', 0], 18: ['Denmark', 0], 19: ['Ecuador', 0], 20: ['Eritrea', 0], 21: ['Albania', 0], 22: ['Bahamas', 0], 23: ['Barbados', 0], 24: ['Armenia', 0], 25: ['Brazil', 0], 26: ['Argentina', 0], 27: ['Croatia', 0], 28: ['Georgia', 0], 29: ['Grenada', 0], 30: ['Egypt', 0], 31: ['Belarus', 0], 32: ['Antigua and Barbuda', 0], 33: ['Cambodia', 0], 34: ['Belize', 0], 35: ['Chile', 0], 36: ['Bahrain', 0], 37: ['Benin', 0], 38: ['Bangladesh', 0], 39: ['Dominica', 0], 40: ['Eswatini', 0], 41: ['Fiji', 0], 42: ['Brunei', 0], 43: ['Afghanistan', 0], 44: ['Costa Rica', 0], 45: ['Greece', 0], 46: ['Chad', 0], 47: ['

In [1]:
import folium

loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=[27.783889, -97.510556],
               zoom_start=12)

m.get_root().html.add_child(folium.Element(title_html))

m.save('map-with-title.html')
m

In [99]:
print(clusters)

[{'Azerbaijan', 'Guinea-Bissau', 'Botswana', 'Finland', 'Greenland', 'Colombia', 'Gabon', 'Dominican Republic', 'Canada', "Cote d'Ivoire", 'Algeria', 'Equatorial Guinea', 'Comoros', 'Haiti', 'Cameroon', 'Hungary', 'Central African Republic', 'Burma', 'Denmark', 'Ecuador', 'Eritrea', 'Albania', 'Bahamas', 'Barbados', 'Armenia', 'Brazil', 'Argentina', 'Croatia', 'Georgia', 'Grenada', 'Egypt', 'Belarus', 'Antigua and Barbuda', 'Cambodia', 'Belize', 'Chile', 'Bahrain', 'Benin', 'Bangladesh', 'Dominica', 'Eswatini', 'Fiji', 'Brunei', 'Afghanistan', 'Costa Rica', 'Greece', 'Chad', 'Honduras', 'Djibouti', 'Cyprus', 'Austria', 'Gambia', 'Ethiopia', 'Guatemala', 'Bosnia and Herzegovina', 'Guinea', 'Burkina Faso', 'Congo (Brazzaville)', 'Guyana', 'Czechia', 'Cuba', 'Holy See', 'Andorra', 'Estonia', 'Angola', 'Burundi', 'Bulgaria', 'Iceland', 'El Salvador', 'Ghana', 'Belgium', 'Bhutan', 'Congo (Kinshasa)', 'Bolivia', 'Cabo Verde'}, {'Madagascar', 'Qatar', 'Netherlands', 'Slovakia', 'Poland', 'Nig

In [ ]:

'''


def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)


geolocator = Nominatim(user_agent="anna_visualization_for_covid")
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

df['codes']=df['CountryName'].apply(get_continent)
df['Country']=df['codes'].apply(lambda pair: pair[0])
df['Continent']=df['codes'].apply(lambda pair: pair[1])

# geopandas doesn't work (( SORRY!!!
import pyproj
pyproj.Proj("+init=epsg:4326")
pyproj.pyproj_datadir


geo_df=geopandas.read_file(country_shapes)
merged_df = geo_df.merge(right=df, right_on='CountryName', left_on='ADMIN', how='left').fillna(np.nan)

gdf = geopandas.GeoDataFrame(geo_df, geometry=geo_df['geometry'])
gdf.crs = {'init' :'epsg:2154'}
gdf = gdf.to_crs({'init' :'epsg:4326'}) 


#df['Geolocate']=df['Country'].apply(geolocate)
#df['latitude']=df['Geolocate'].apply(lambda pair: pair[0])
#df['longitude']=df['Geolocate'].apply(lambda pair: pair[1])

#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'


country_shapes = f'{url}/world-countries.json'
m= folium.Map(tiles="cartodbpositron")
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='choropleth COVID-19',
    data=df,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['CountryName', 'Cluster'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    nan_fill_color='grey'
).add_to(m)
m.save('covid.html')


#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df)):
        #lat = df.iloc[i]['Geolocate'][0]
        #long = df.iloc[i]['Geolocate'][1]
        radius=8
        popup_text = """Country : {}<br>
                    %of Cluster : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Country'],
                                   df.iloc[i]['Cluster']
                                   )
        folium.CircleMarker(location = list(df.iloc[i]['Geolocate']), radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map.save('my_map.html')


#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'


country_shapes = f'{url}/world-countries.json'
m= folium.Map(tiles="cartodbpositron")
folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='choropleth COVID-19',
    data=df,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['CountryName', 'Cluster'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    nan_fill_color='white'
).add_to(m)
m.save('covid.html')


'''